# US 2022 Stock Market Analysis

In this notebook, I will attempt to investigate how the various sectors of the US economy performed on the stock market in 2022 by creating a market cap-weighted index of each sector, and plotting its returns over the year.

The stocks are classified into these sectors according to the Global Industry Classification  Standard (GICS):
- Communication Services
- Consumer Discretionary
- Consumer Staples
- Energy
- Financials
- Healthcare
- Industrials 
- Information Technology
- Materials
- Real Estate
- Utilities


In [10]:
import yfinance as yf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

### Downloading stock data

I first had to get a table of all the stocks currently trading in the US. I downloaded the data below from the [NASDAQ Stock Screener](https://www.nasdaq.com/market-activity/stocks/screener).

In [11]:
# Data from NASDAQ stock screener
us_stocks = pd.read_csv('us_stocks.csv')

In [12]:
%%capture
us_stocks

In [13]:
us_stock_ticker_symbols = us_stocks['Symbol']

Next, I downloaded the 2022 price history of each stock, saving each in its own file in the folder `us_stocks`.
This price data is from [Yahoo Finance](https://finance.yahoo.com), accessed through the `yfinance` library.

In [14]:
%%capture
tickers_loaded = []
tickers_not_loaded = []
for i in us_stock_ticker_symbols:
    ticker = i.replace('^', '-P')
    # Make sure there is no error associated with downloading the stock data
    try:
        data = yf.download(ticker, start='2022-01-01', end='2022-12-31')
    except:
        tickers_not_loaded += [ticker]
        continue
    # Make sure ticker was traded on all trading days in 2022
    # Also takes care of tickers that failed to download but did not raise an exception
    if len(data) != 251:
        tickers_not_loaded += [ticker]
        continue
    # / character is not allowed in file names
    file_name = ticker.replace('/','{slash}')
    data.to_csv(f'stock_data/{file_name}.csv')
    tickers_loaded += [ticker]

In [ ]:
# Data of tickers that managed to load and tickers that did not. Needs to be changed to DataFrame to make using it easier
tickers_loaded = pd.DataFrame(tickers_loaded, columns=['Symbol'])
tickers_not_loaded = pd.DataFrame(tickers_not_loaded, columns=['Symbol'])

I need to replace '^' (found in NASDAQ stock tickers) with '-P' (found in Yahoo finance stock tickers). I also need to replace any '/' found in the ticker name with '{slash}' (as / is not allowed in file name)>

In [ ]:
us_stocks['Symbol'] = us_stocks['Symbol'].replace({'\^': '-P'}, regex=True)
us_stocks['Symbol'] = us_stocks['Symbol'].replace({'/': '{slash}'}, regex=True)

### Cleaning up data (mostly renaming of Sectors

In this section, I clean up the data, making sure each stock has a Sector and Market Cap, and the Sector names are in line with the Global Industry Classification Standard (GICS).

In [ ]:
# The breakdown of sectors according to NASDAQ
sectors = pd.DataFrame(us_stocks.groupby('Sector').count().index)

In [ ]:
sectors

In [ ]:
us_stocks[us_stocks['Sector']=='Miscellaneous']

In [ ]:
indexes_to_drop = list(us_stocks[us_stocks['Symbol'].isin(tickers_not_loaded['Symbol'])].index)

In [ ]:
us_stocks.drop(indexes_to_drop, inplace=True)

In [ ]:
# checking which tickers have a sector marked as Miscellaneous
us_stocks[us_stocks['Sector']=='Miscellaneous']

In [ ]:
# checking which ickers have missing sectors
us_stocks[us_stocks['Sector'].isna()]

In [ ]:
# Attempt to assign sector to tickers that have no sector
for i in range(len(us_stocks)):
    if pd.isnull(us_stocks.iloc[i]['Sector']):
        # check if it is a variant of an existing stock
        if ('-P' in us_stocks.iloc[i]['Symbol']) & (us_stocks.iloc[i]['Symbol'].split('-')[0] in us_stocks['Symbol'].values ):
            us_stocks.at[i,'Sector'] = str(us_stocks[us_stocks['Symbol']==us_stocks.iloc[i]['Symbol'].split('-')[0]]['Sector'].values[0])
            continue

        # get  sector classification from yahoo finance
        stock_yf_data = yf.Ticker(us_stocks.iloc[i]['Symbol']).info
        if 'sector' in stock_yf_data.keys():
            us_stocks.at[i,'Sector'] = stock_yf_data['sector']



In [ ]:
# The stocks with sectors still missing
us_stocks[us_stocks['Sector'].isna() | us_stocks['Sector']=='Miscellaneous']

In [ ]:
# Mannually replacing the remaining missing sectors
us_stocks.at[83,'Sector'] = 'Industrials'
us_stocks.at[102,'Sector'] = 'Utilities' # check if this is true
us_stocks.at[292,'Sector'] = 'Finance'
us_stocks.at[391,'Sector'] = 'Technology'
us_stocks.at[498,'Sector'] = 'Industrials'
us_stocks.at[639,'Sector'] = 'Consumer Discretionary'
us_stocks.at[931,'Sector'] = 'Industrials'
us_stocks.at[1120,'Sector'] = 'Consumer Discretionary'
us_stocks.at[1774,'Sector'] = 'Industrials'
us_stocks.at[1787,'Sector'] = 'Industrials'
us_stocks.at[1794,'Sector'] = 'Industrials'
us_stocks.at[1795,'Sector'] = 'Industrials'
us_stocks.at[2033,'Sector'] = 'Utilities'
us_stocks.at[2039,'Sector'] = 'Industrials'
us_stocks.at[2093,'Sector'] = 'Consumer Discretionary'
us_stocks.at[2094,'Sector'] = 'Consumer Discretionary'
us_stocks.at[2402,'Sector'] = 'Industrials'
us_stocks.at[2887,'Sector'] = 'Health Care'
us_stocks.at[2958,'Sector'] = 'Health Care'
us_stocks.at[3050,'Sector'] = 'Consumer Discretionary'
us_stocks.at[3058,'Sector'] = 'Health Care'
us_stocks.at[3571,'Sector'] = 'Technology'
us_stocks.at[3592,'Sector'] = 'Industrials'
us_stocks.at[3621,'Sector'] = 'Consumer Discretionary'
us_stocks.at[3692,'Sector'] = 'Utilities'
us_stocks.at[3721,'Sector'] = 'Industrials'
us_stocks.at[3758,'Sector'] = 'Industrials'
us_stocks.at[3968,'Sector'] = 'Technology'
us_stocks.at[3993,'Sector'] = 'Industrials'
us_stocks.at[4186,'Sector'] = 'Industrials'
us_stocks.at[4542,'Sector'] = 'Technology'
us_stocks.at[4612,'Sector'] = 'Consumer Discretionary'
us_stocks.at[4874,'Sector'] = 'Consumer Staples'
us_stocks.at[4986,'Sector'] = 'Technology'
us_stocks.at[5335,'Sector'] = 'Industrials'
us_stocks.at[5456,'Sector'] = 'Technology'
us_stocks.at[5625,'Sector'] = 'Consumer Discretionary'
us_stocks.at[5716,'Sector'] = 'Consumer Discretionary'
us_stocks.at[99,'Sector'] = 'Financials'
us_stocks.at[121,'Sector'] = 'Financials'
us_stocks.at[1286,'Sector'] = 'Energy'
us_stocks.at[1593,'Sector'] = 'Utilities'
us_stocks.at[2529,'Sector'] = 'Financials'
us_stocks.at[3163,'Sector'] = 'Utilities'
us_stocks.at[3296,'Sector'] = 'Financials'
us_stocks.at[3297,'Sector'] = 'Financials'
us_stocks.at[4018,'Sector'] = 'Utilities'
us_stocks.at[4021,'Sector'] = 'Utilities'
us_stocks.at[4022,'Sector'] = 'Financials'

In [ ]:
us_stocks[us_stocks['Sector'].isna() | us_stocks['Sector']=='Miscellaneous']  # This confirms that no ticker has a missing sector

In [ ]:
sectors = pd.Series(us_stocks.groupby('Sector').count().index)
sectors

In [ ]:
# Rename sectors to GICS sector names
us_stocks['Sector'].replace('Basic Materials','Materials', inplace=True)
us_stocks['Sector'].replace('Health Care','Healthcare', inplace=True)
us_stocks['Sector'].replace('Finance','Financials', inplace=True)
us_stocks['Sector'].replace('Technology','Information Technology', inplace=True)
us_stocks['Sector'].replace('Telecommunications','Communication Services', inplace=True)

In [ ]:
sectors = pd.Series(us_stocks.groupby('Sector').count().index)
sectors 

In [ ]:
# Rename sectors to their GICS names
us_stocks['Sector'].replace('Consumer Cyclical','Consumer Discretionary', inplace=True)
us_stocks['Sector'].replace('Consumer Defensive','Consumer Staples', inplace=True)
us_stocks['Sector'].replace('Financial Services','Financials', inplace=True)
us_stocks['Sector'].replace('Technology','Information Technology', inplace=True)
us_stocks['Sector'].replace('Basic Materials','Materials', inplace=True)

In [ ]:
# Now to clean up the market capitalisation data
us_stocks[us_stocks['Market Cap'].isna()]

In [ ]:
# To make things easier we're going to use the Market Cap of the stock today (instead of in 2022). This means the indexes won;t be entirely accurate, but they should be close enough
# For the stocks with NaN Market Caps, we will try download market cap data from Yahoo Finance
for i in range(len(us_stocks)):
    if pd.isnull(us_stocks.iloc[i]['Market Cap']):

        # get market cap from yahoo finance (if data exists)
        stock_yf_data = yf.Ticker(str(us_stocks.iloc[i]['Symbol'])).info
        if 'marketCap' in stock_yf_data.keys():
            us_stocks.at[i,'Market Cap'] = stock_yf_data['marketCap']
            continue

        #if there's no data on yahoo finance, get average market cap of the sector
        stock_sector = us_stocks.iloc[i]['Sector']
        us_stocks.at[i,'Market Cap'] = us_stocks[us_stocks['Sector']==stock_sector]['Market Cap'].mean()

For testing purposes, `us_stocks`, I calculate the mean Market Cap:

In [ ]:
us_stocks[(us_stocks['Sector']=='Financials') & (~us_stocks['Market Cap'].isna())]['Market Cap'].mean() #the mean market capitalisation

In [ ]:
us_stocks.drop(['level_0','Unnamed: 0','index'],axis=1, inplace=True)

Any rows with missing items are dropped at this point - I haven't been able to find the data.

In [ ]:
us_stocks.dropna(inplace=True)

### Starting off - building an S&P 500 Equal Weight Index, and S&P500 normal index

I start off with building an S&P500 Equal Weight Index (for simplicity). This is the S&P 500 index, but with each stock having an equal weighting in the index.

In [ ]:
# Add a percentage change column to all stock csvs in stock_data, and drop unnecessary columns
successful = []
for i in us_stocks['Symbol']:
    stock_data = pd.read_csv(f'stock_data/{i}.csv')
    stock_data.drop(['Open','High','Low','Close','Volume'], axis=1, inplace=True)
    stock_data['% change'] = stock_data['Adj Close'].pct_change()
    stock_data.at[0,'% change'] = 0
    stock_data.to_csv(f'stock_data/{i}.csv')
    successful += [i]

For testing purposes, I plot the daily percentage change in Google's GOOG stock, for 2022:

In [ ]:
pd.read_csv('stock_data/GOOG.csv')['% change'].plot() # Google's daily returns plot

A dataframe of S&P 500 tickers is scraped from a wikipedia article:

In [ ]:
# Get dataframe of tickers in the S&P500 index
snp_ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol']]

In [ ]:
snp_ticker_list['Symbol']

Then I isolate the S&P 500 tickers I managed to download (and managed to fill in missing data):

In [ ]:
exists = []
for i in snp_ticker_list['Symbol']:
    print(i)
    if os.path.exists(f'stock_data/{i}.csv'):
        exists += [i]

In [ ]:
# These are the tickers that are in the S&P500, but for some reason, had to be dropped. We will disregard these to make the project easier
snp_ticker_list[~snp_ticker_list['Symbol'].isin(exists)]


`snp_weights` is a list of the tickers' relative weights. As this is an Equal Weight Index, each stock has an equal weighting:

In [ ]:
snp_weights = np.repeat(1/len(exists),len(exists)) # creating a list of weights, with each stock having equal weighting in this case

`snp_index` contains the daily returns of each stock in the index, on each trading day in 2022:

In [ ]:
snp_index = pd.DataFrame()
for i in exists:
    snp_index[i] = pd.read_csv(f'stock_data/{i}.csv')['% change']


The `index_returns` variable is added to `snp_index`, and this is the calculation of the index's returns on each trading day in 2022:

In [ ]:
snp_index['index_returns'] = snp_index.mul(snp_weights,axis=1).sum(axis=1)
snp_index

The index returns are converted to cumulative returns to give the total returns through time

In [ ]:
snp_index['cumulative returns'] = ((1+snp_index['index_returns']).cumprod()-1)

And the result is plotted. The resulting plot is very similar to the actual S&P 500 EWI graph, indicating that the tickers that had to be excluded did not have a major effect on the final result.

In [ ]:
snp_index['cumulative returns'].plot()
# Quite close to actual graph but not perfect - year change is about 1 percentage point from the actual value

I then follow the same steps to create the actual S&P 500 (weighted) index:

In [ ]:
# To create a market cap weighted index
snp_weights = []
for i in exists:
    mktcap = float(us_stocks[us_stocks['Symbol']==i]['Market Cap'])
    snp_weights += [mktcap/us_stocks[us_stocks['Symbol'].isin(exists)]['Market Cap'].sum()]
snp_index.drop(['index_returns','cumulative returns'], axis=1, inplace=True)
snp_index['index_returns'] = snp_index.mul(snp_weights,axis=1).sum(axis=1)
snp_index['cumulative returns'] = ((1+snp_index['index_returns']).cumprod()-1)
snp_index['cumulative returns'].plot()

In [ ]:
snp_index['cumulative returns'].iloc[250] # actual return is -0.1944, so this one is about 2 percentage points from the actual

### Next Step: making an index of each industries in the US

In [ ]:
sectors

In [ ]:
# compute the different weights of each industry based on market capitalisation
industry_weights = {}
for i in sectors:
    weights = []
    industry_tickers = list(us_stocks[us_stocks['Sector'] == i]['Symbol'])

    for j in industry_tickers:
        market_cap = float(us_stocks[us_stocks['Symbol']==j]['Market Cap'])
        weights += [market_cap/us_stocks[us_stocks['Symbol'].isin(industry_tickers)]['Market Cap'].sum()]
    industry_weights[i] = weights

In [ ]:
# Compute the cumulative returns of all sectors
industry_return_data = pd.DataFrame()
for i in sectors:
    return_data = pd.DataFrame()
    industry_tickers = list(us_stocks[us_stocks['Sector'] == i]['Symbol'])
    for j in industry_tickers:
        return_data[j] = pd.read_csv(f'stock_data/{j}.csv')['% change']
    return_data['index_returns'] = return_data.mul(industry_weights[i],axis=1).sum(axis=1)
    return_data['cumulative returns'] = ((1+return_data['index_returns']).cumprod()-1)
    industry_return_data[i] = return_data['cumulative returns'] # this is all we need. We can throw out the rest

In [ ]:
industry_return_data.plot()

## Conclusion